<a href="https://colab.research.google.com/github/randywreed/DigiJesus-GPT3/blob/main/digitaljesus_prompt_gpt_neo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 6.5MB/s 
     |████████████████████████████████| 3.3MB 46.8MB/s 
     |████████████████████████████████| 901kB 39.4MB/s 


In [2]:
import os
import transformers


In [ ]:
#if not using gdrive
modelpath="EleutherAI/gpt-neo-1.3B"
tokenpath="EleutherAI/gpt-neo-1.3B"

In [3]:
#skip if not using Gdrive 
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#if using Gdrive (probably requires colab pro get's out of memory error)
modelpath='/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomodel/model'
tokenpath='/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomodel/tokenizer'


In [5]:
# download pretrained model and save to google drive. 
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
model.save_pretrained(modelpath)
tokenizer.save_pretrained(tokenpath)

('/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomode/tokenizer/tokenizer_config.json',
 '/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomode/tokenizer/special_tokens_map.json',
 '/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomode/tokenizer/vocab.json',
 '/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomode/tokenizer/merges.txt',
 '/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomode/tokenizer/added_tokens.json')

In [3]:
from transformers import pipeline
generator = pipeline('text-generation', model=modelpath, tokenizer=tokenpath, device=0)


file /content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/gptneomodel/tokenizer/config.json not found


OSError: ignored

In [ ]:


with open("/content/gdrive/MyDrive/Data Analysis for Humanities/Jesus saying database/colab_exp/jesus_prompts.txt","r+") as promptfile:
    initprompt=promptfile.read()
    newpassflag=True
    while newpassflag:
        newpassage=input('enter new passage: ')
        prompt=initprompt+"\npassage:"+newpassage+"\nquote:"
        print(prompt[-len(newpassage)+15])
        loopflag=True
        while loopflag:        
            minl=len(prompt)+100
            print(minl)
            response=generator(geninput, do_sample=True, min_length=len(geninput),max_length=minl)
            print(response)
            action=input('type correction, hit return to retry, a to add, x to stop, r to re-enter: ')
            if len(action)>1:
                promptfile.write(action+"\n###\n")
            else:
                if action=="a":
                    newresponse=response.choices[0].text.split("###")
                    promptfile.write("\npassage: "+newpassage+"\nquote:"+ newresponse[0]+"###")
                    loopflag=False
                    getmore=input('enter c to continue or x to exit')
                    if getmore=="x":
                        promptfile.close()
                        exit()
                else:
                    if action=="x":
                        promptfile.close()
                        exit()
                    else:
                        if action=="r":
                            loopflag=False